In [1]:
import numpy as np
import math
import pandas as pd
from scipy.stats import norm
from scipy.integrate import trapz
from IPython.display import display, clear_output
from cvxopt import matrix, solvers
import datetime, time, itertools
from collections import defaultdict
from scipy.interpolate import CubicSpline
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
from bokeh.plotting import figure
import bokeh.models as bom
from bokeh.models import LinearAxis, Range1d
import quandl
from tqdm import tqdm_notebook
import warnings
from pulp import LpProblem, LpMaximize, LpVariable, LpAffineExpression, LpConstraint, value
warnings.filterwarnings('ignore')
output_notebook()

Loading BokehJS ...

In [2]:
#Cubic Spline modeling on curve fitting
def cubicspline_model(xs, zs, vols, cs):
    res = np.zeros(xs.shape)
    res[np.logical_and(xs<=zs[-1], xs>=zs[0])] = cs(xs[np.logical_and(xs<=zs[-1], xs>=zs[0])])
    res[xs>zs[-1]] = cs(zs[-1], 1)*(xs[xs>zs[-1]]-zs[-1]) + vols[-1]
    res[xs<zs[0]] = cs(zs[0], 1)*(xs[xs<zs[0]]-zs[0]) + vols[0]
    return res

class ZCubicSpline():
    def __init__(self, zs, vols, vol_cap=[0.03, 1.60], z_bound=[-6,6], num_k=3e3):
        self.vol_cap = vol_cap
        self.xs = np.linspace(z_bound[0], z_bound[1], num_k)
        self.ys = cubicspline_model(self.xs, zs, vols, CubicSpline(zs, vols, bc_type='natural', extrapolate=False))
    def evaluate(self, k, f, t):
        ks = f*np.exp(self.xs*self.ys*np.sqrt(t)-0.5*np.power(self.ys,2)*t)
        return np.minimum(np.maximum(np.interp(k, ks, self.ys), self.vol_cap[0]), self.vol_cap[1])
    def evaluate_z(self, z):
        return np.minimum(np.maximum(np.interp(z, self.xs, self.ys), self.vol_cap[0]), self.vol_cap[1])
    def calc_k(self, z, f, t):
        return f*np.exp(z*self.evaluate_z(z)*np.sqrt(t)-0.5*np.power(self.evaluate_z(z),2)*t)
    def calc_z(self, k, f, t):
        return np.log(k/f)/self.evaluate(k,f,t)/np.sqrt(t) + 0.5*self.evaluate(k,f,t)*np.sqrt(t)

In [3]:
# Option price/ greeks calculation
class Option:
    """
    This class will group the different black-scholes calculations for an opion
    """
    def __init__(self, option_type, s, k, eval_date, exp_date, price = None, rf = 0.0175):
        """
        option_type: C or P, strike - k, forward - s
        """
        self.k = float(k)
        self.s = float(s)
        self.rf = float(rf)
        self.eval_date = eval_date
        self.exp_date = exp_date
        self.t = max(self.calculate_t(), 0.00001) #time to expiry
        self.price = price
        self.type = option_type   # 'C' or 'P'
        self.div = 0 
    
    def calculate_t(self):
        """
        Calculate time to expiry
        """
        return (self.exp_date - self.eval_date).days / 365.0

    def get_price_delta(self):
        d1 = ( math.log(self.s/self.k) + ( self.rf + self.div + math.pow( self.vol, 2)/2 ) * self.t ) / ( self.vol * math.sqrt(self.t) )
        d2 = d1 - self.vol * math.sqrt(self.t)
        if self.type == 'C':
            self.calc_price = ( norm.cdf(d1) * self.s * math.exp(-self.div*self.t) - norm.cdf(d2) * self.k * math.exp( -self.rf * self.t ) )
            self.delta = norm.cdf(d1)
        elif self.type == 'P':
            self.calc_price = ( -norm.cdf(-d1) * self.s * math.exp(-self.div*self.t) + norm.cdf(-d2) * self.k * math.exp( -self.rf * self.t ) )
            self.delta = -norm.cdf(-d1) 
    
    def get_impl_vol(self):
        """
        This function will iterate until finding the implied volatility
        """
        ITERATIONS = 1000
        ACCURACY = 0.005
        low_vol = 0
        high_vol = 2.0
        self.vol = 0.3  # It will try mid point and then choose new interval
        self.get_price_delta()
        for i in range(ITERATIONS):
            if self.calc_price > self.price * (1 + ACCURACY):
                high_vol = self.vol
            elif self.calc_price < self.price * (1 - ACCURACY):
                low_vol = self.vol
            else:
                break
            self.vol = low_vol + (high_vol - low_vol)/2.0
            self.get_price_delta()
 
        return self.vol

In [4]:
def find_strike_from_delta(delta, ks, atm_vol, f, tte):
    """
    Calculate corresponding strikes that are closest to strike list (ks) 
    matching the delta bracket.
    """
    r = 0
    d1_list = np.vectorize(norm.ppf)(delta / 100)
    k_list = f / np.exp(d1_list * atm_vol * np.sqrt(tte) - (r + (tte**2)/2) * tte)
    k_list = [ks[np.absolute(k - ks).argmin()] for k in k_list] 
    return k_list[::-1]

def lognormal_dist(sigma, f, t):
    """
    Generate log normal distribution function from volatility (sigma), atm future(f), and vol time
    """
    def func(x):
        norm_input = ((np.log(x/f) + 0.5 * sigma**2 * t)/ (sigma * np.sqrt(t)))
        norm = 1/np.sqrt(2*np.pi)*np.exp(-0.5*np.power(norm_input, 2))
        return norm/ sigma/ np.sqrt(t)/ x
    return func

# Download ES End of Month Option settlement price on 2018/05/11 for June 2018 expiry

In [5]:
# Download ES option & future data
quandl.ApiConfig.api_key = 'FYVALwRtpJkfmtyysPTR'
today = '2018-05-11'
ES_option_price = quandl.get_table('AR/OWCS', code='CME_EW_M2018_S', date = today, paginate=True)
ES_future = quandl.get('CME/ESM2018', start_date=today, end_date=today)['Settle'].iloc[0]

# Calculate Forward and ATM vol

In [6]:
option_strip = ES_option_price
strike_list = np.sort(option_strip.strike.unique())
ATM_strike = strike_list[np.argmin(np.abs(strike_list - ES_future))]
ATM_call = option_strip[(option_strip['strike'] == ATM_strike) & (option_strip['type'] == 'C')].settlement.iloc[0]
ATM_put = option_strip[(option_strip['strike'] == ATM_strike) & (option_strip['type'] == 'P')].settlement.iloc[0]

f = forward = ATM_call - ATM_put + ATM_strike # Find forward value from ATM put call parity
last_trading_date = '2018-06-29'

#Get ATM vol
ATM_call_option = Option(option_type='C', s=forward, k=ATM_strike, eval_date= pd.to_datetime(today), exp_date=pd.to_datetime(last_trading_date), price = ATM_call)
ATM_put_option = Option(option_type='P', s=forward, k=ATM_strike, eval_date= pd.to_datetime(today), exp_date=pd.to_datetime(last_trading_date), price = ATM_put)
ATM_vol = 0.5*ATM_call_option.get_impl_vol() + 0.5*ATM_put_option.get_impl_vol() #Take average as implied vol

# Calculate implied vol given a set of delta bucket

In [7]:
delta_list = np.array([0.1, 1, 5, 20, 30, 40, 50, 60, 70, 80, 95, 99, 99.9])
t = ATM_call_option.calculate_t() # Time to expiry
k_list = find_strike_from_delta(delta_list, strike_list, ATM_vol, forward, t)
vol_list = []
for k in k_list:
    option_k = option_strip[(option_strip['strike'] == k)]
    if len(option_k) != 2:
        continue

    call_price = option_k[(option_k['type'] == 'C')].settlement.iloc[0]
    call_option = Option(option_type='C', s=forward, k=k, eval_date=pd.to_datetime(today), exp_date=pd.to_datetime(last_trading_date), price = call_price)
    call_vol = call_option.get_impl_vol() 
    
    put_price = option_k[(option_k['type'] == 'P')].settlement.iloc[0]
    put_option = Option(option_type='P', s=forward, k=k, eval_date=pd.to_datetime(today), exp_date=pd.to_datetime(last_trading_date), price = put_price)
    put_vol = put_option.get_impl_vol() 
    
    buffer = 30
    if k <= ATM_strike - buffer:
        vol = put_vol
    elif ATM_strike - buffer < k <  ATM_strike + buffer:
        vol = (put_vol + call_vol) / 2
    else:
        vol = call_vol
    vol_list.append(vol)
vol_list = np.array(vol_list)

# Fit cubic spline implied vol curve

In [8]:
#Fit cubic spline on implied vol surface
cs = CubicSpline(k_list, vol_list, bc_type='natural', extrapolate=True)
ks = np.linspace(k_list[0], k_list[-1], 100)

In [9]:
fig = figure(width=600, height=300, x_axis_label='strike', y_axis_label='i-vol')
fig.scatter(x=k_list, y=vol_list, legend='Benchmark', color='red', line_width=1)
fig.line(x=ks, y=cs(ks), legend='Vol Curve', color='green', line_width=2)
fig.legend.location = 'top_right'
show(fig)

In [10]:
# Correspondence between strikes, vols, and zs
vol_min_bound = 0.03
vol_max_bound = 1.6
vols = np.maximum(cs(ks), vol_min_bound)
zs = np.log(ks/f)/vols/np.sqrt(t) + 0.5 * vols * np.sqrt(t)
z_mask = np.logical_and(zs > -3, zs < 3)
k_z_vol = np.vstack([ks, zs, vols]).T[z_mask]

sp_zs = np.linspace(-2.5, 2.5, 21)
sp_kzv = np.array([k_z_vol[np.absolute(zs - z).argmin()] for z in sp_zs])
sp_kzv = sp_kzv[np.unique(sp_kzv[:, 0], return_index=True)[1]]
sp_ks, sp_zs, sp_vols = sp_kzv[:,0], sp_kzv[:,1], sp_kzv[:,2]
sp_cs = ZCubicSpline(sp_zs, sp_vols, (vol_min_bound, vol_max_bound))

# Convert to Z_space (standard deviations) Implied vol curve

In [11]:
fig = figure(width=600, height=300, x_axis_label='z_node', y_axis_label='i-vol')
fig.line(x=sp_zs, y=sp_vols, legend='Z-node Vol Curve', color='blue', line_width=2)
fig.legend.location = 'top_right'
show(fig)

# VCR (Volatility Change Rate) measures how ATM vol change w.r.t. future change

> ## <font color = darkblue>VCR = $\frac{d(ATMvol)}{d(\ln(f))}$</font>


In [12]:
"""
VCR & NCR parameter Config
"""
vcr_params = {
    'base_coef': 1.38,
    'intra_day': -10.0,
    'prev_day': -6.0,
    'over_night': -10.0
}

ncr_params = {
    'base_ncr': {-2.5: 0.84, -2.0: 0.92, -1.5: 1.0, -1.0: 1.04, -0.5: 1.02, 0.0: 1.0,
              0.5: 0.94, 1.0: 0.87, 1.5: 0.75, 2.0: 0.6, 2.5: 0.3},
    'z_adj': {-2.5: 0.08, -1.5: 0.24, 2.5: -0.4},
    'f_adj': {-0.04: -2.0, 0.0: 0.0, 0.04: 1.0}
}

val = forward
today_open = 2730
prev_close = 2725
prev_open = 2720

day_move, prev_move, overnight_move = np.log(val/today_open), np.log(prev_close/prev_open), \
                                              np.log(today_open/prev_close) 
trends = (day_move, prev_move, overnight_move)

In [13]:
def live_ncr_func(val, base_ncr, ncr_z_adj, ncr_f_adj):
    """
    Return a function that provides the total (average) ncr for a certain z-node given a future scenario
    """
    zs = sorted(base_ncr.keys())
    ncr = [base_ncr[z] for z in zs]
    
    z_adj = sorted(ncr_z_adj.keys())
    z_coef = [ncr_z_adj[z] for z in z_adj]
    
    f_adj = sorted(ncr_f_adj.keys())
    f_coef = [ncr_f_adj[f] for f in f_adj]
    
    def func(z, f):
        dln_atmf = np.log(f/val)
        adj_ncr = np.interp(z, zs, ncr) + np.interp(z, z_adj, z_coef) * np.interp(dln_atmf, f_adj, f_coef)
        return adj_ncr
    return func

In [14]:
def calculate_base_vcr(base_val, vcr_params, trends):
    day_move, prev_move, overnight_move = trends
    base_vcr =  vcr_params['base_coef'] + \
                vcr_params['intra_day'] * day_move + \
                vcr_params['prev_day'] * prev_move + \
                vcr_params['over_night'] * overnight_move
    return base_vcr

def calculate_vcr(base_vcr, val, time, sp_cs):
    """
    Calculate vcr by adding skew factors to base_vcr
    """
    time = np.maximum(time, 0.05/256)
    dvdz = sp_cs.evaluate_z(0.5) - sp_cs.evaluate_z(-0.5)
    center_node_vol = sp_cs.evaluate_z(0)
    f_t = dvdz / center_node_vol / np.sqrt(time)
    return f_t * base_vcr, f_t

def live_vcr_func(val, base_vcr, vcr_day_beta):
    """
    Return a function that provides the total (average) vcr given a future scenario
    """
    def func(f):
        adj_vcr = base_vcr + 0.5 * vcr_day_beta * np.log(f/val)
        return adj_vcr
    return func

def calc_new_vol(val, perc_change, vol, vcr_func):
    """
    Return new vol given perc_change future scenario
    """
    f_scenario = val * np.exp(perc_change)
    new_vol = vol + vcr_func(f_scenario) * perc_change
    return new_vol

In [15]:
base_vcr = calculate_base_vcr(val, vcr_params, trends)
vcr, f_t = calculate_vcr(base_vcr, val, t, sp_cs)
vcr_day_beta = f_t * (vcr_params['intra_day'])
vcr_func = live_vcr_func(val, vcr, vcr_day_beta)
ATM_vol = sp_cs.evaluate_z(0)

# Test new vol prediction given future scenario

In [16]:
f_scenario = 2700
new_vol = ATM_vol + vcr_func(f_scenario) * np.log(f_scenario / val)
print('New vol is {} given future level of {}'.format(round(new_vol, 4), f_scenario))

New vol is 0.1263 given future level of 2700


In [17]:
# Calculate a joint list of 101 future val scenarios for optimization and 
# set center node future to current val
offset = 0
scen_zs = np.round(np.linspace(-4, 4, 101), 2)
fs = sp_cs.calc_k(scen_zs, f, t)
fs[scen_zs == 0] = f
base_fs = fs - offset

# Create a fine approximation for a log-normal distribution, precompute densities
log_dist = lognormal_dist(sp_cs.evaluate_z(0), f, np.clip(t, 1e-6, 1/256))
base_xn = np.linspace(base_fs[0], base_fs[-1], int(1e4))
full_dist = log_dist(base_xn)

#Handle offset between expiries
fs = base_fs + offset
xn = base_xn + offset

In [18]:
scen_zs_vol = np.vectorize(calc_new_vol)(val, scen_zs/100, ATM_vol, vcr_func)

In [19]:
fig = figure(width=600, height=300, x_axis_label='Future change perc (%)', y_axis_label='i-vol')
fig.line(x=scen_zs, y=scen_zs_vol, legend='ATM vol under future scenario', color='blue', line_width=2)
fig.legend.location = 'top_right'
show(fig)

# NCR (Node Change Rate) measures how other areas of the curve change relative to ATM vol change
- NCR on a particular z_node is defined as:
    
> ## <font color = darkblue> $\ln(\frac{New\_vol(z)}{Old\_vol(z)})$ = $NCR(z)$ $\times$ $\ln(\frac{New\_vol(atm)}{Old\_vol(atm)})$</font>

In [20]:
ncr_func = live_ncr_func(val, ncr_params['base_ncr'], ncr_params['z_adj'], ncr_params['f_adj'])

# Test NCR with future scenario

In [21]:
f_scenario = 2780
new_center_node = calc_new_vol(val, np.log(f_scenario/val), ATM_vol, vcr_func)
ncr_z = np.vectorize(ncr_func)(sp_zs, f_scenario)
new_z_vol = np.power(new_center_node/ATM_vol, ncr_z) * sp_vols

In [22]:
fig = figure(width=600, height=300, x_axis_label='z_node', y_axis_label='i-vol')
fig.line(x=sp_zs, y=sp_vols, legend='Old scenario Z-node Vol Curve', color='blue', line_width=2)
fig.line(x=sp_zs, y=new_z_vol, legend='New scenario Z-node Vol Curve', color='red', line_width=2)
fig.legend.location = 'top_right'
show(fig)

# Model future distribution as lognormal

In [23]:
# Lognormal Future movement distribution

fig = figure(width=600, height=300, x_axis_label='future', y_axis_label='distribution')
fig.line(x=base_xn, y=full_dist, legend='Future Distribution', color='blue', line_width=2)
fig.legend.location = 'top_right'
show(fig)

# Generate vol curve on each future scenario based on VCR & NCR prediction

In [24]:
def _vol_generator(val, vcr, ncr, vol_bounds=(0.03, 1.6)):
    def func(fs, zs, vols):
        zm, fm = np.meshgrid(zs, fs)
        center_node_vol = np.interp(0, zs, vols)
        new_center_node_vols = np.clip(center_node_vol + vcr(fm) * np.log(fm/val), vol_bounds[0], vol_bounds[1])
        vol_gen = np.clip(np.power(new_center_node_vols / center_node_vol, ncr(zm, fm)) * vols, vol_bounds[0], vol_bounds[1])
        return vol_gen
    return func

def gen_scen_splines(zs, vols, fs, vol_dynamics, vol_cap=[0.03, 1.60]):
    return [ZCubicSpline(zs, vs, vol_cap) for vs in vol_dynamics(fs, zs, vols)]

In [25]:
#Generate a spline per future scenario to be used for calculations and optimization
vol_bounds = (0.03, 1.6)
vol_grid_func = _vol_generator(val, vcr_func, ncr_func, vol_bounds)
sp_fs = gen_scen_splines(sp_zs, sp_vols, fs, vol_grid_func, vol_bounds)

In [26]:
# Pick strikes evenly spaced in z-space for optimization
z_nodes = np.linspace(-2, 2, 5)
idx = sorted(set([np.absolute(zs-z).argmin() for z in z_nodes]))
opt_kzv = np.array(k_z_vol[idx])
opt_kzv = opt_kzv[np.unique(opt_kzv[:, 0], return_index=True)[1]]
opt_ks, opt_zs, opt_vols = opt_kzv[:, 0], opt_kzv[:, 1], opt_kzv[:, 2]

In [27]:
def _b76_price(vol, k, r, t1, t2, f, opt_1, opt_2):
    d1 = (np.log(f/k) + 1/2 * vol**2 * t1)/(vol * np.sqrt(t1))
    d2 = (np.log(f/k) - 1/2 * vol**2 * t1)/(vol * np.sqrt(t1))
    return f * norm.cdf(d1) - k * norm.cdf(d2)

def b76_vega(f, d, time, exp_mrt=1):
    return exp_mrt * f * np.exp(-d**2 / 2.) * time**0.5 / np.sqrt(2 * np.pi)

def gen_risk_matrix(ks, f0, t, curr_spline, spline_fs, fs, dt=1.0/256):
    km, fm = np.meshgrid(ks, fs)
    next_tte = max(t-dt, 1e-6)
    vm_dt = np.vstack([sp.evaluate(ks, fut, next_tte) for sp, fut in zip(spline_fs, fs)])
    zm_dt = np.log(km / fm) / vm_dt / np.sqrt(next_tte) + 0.5 * vm_dt * np.sqrt(next_tte)
    dm_dt = -zm_dt + vm_dt * np.sqrt(next_tte)
    wm_dt = b76_vega(fm, dm_dt, next_tte)
    
    return zm_dt, wm_dt, vm_dt

In [28]:
#Get list of zs, vegas, vols, center_node_vols under each future scenario
risk_zs, risk_ws, risk_vs = gen_risk_matrix(opt_ks, f, t, sp_cs, sp_fs, fs)
risk_v0 = np.array([sp.evaluate_z(0) for sp in sp_fs])

In [29]:
def gen_dpm(ks, f0, t, curr_spline, spline_fs, fs, dt=1.0/256):
    """
    Generate expected payoff (both hedged and unhedged), delta and theta under different future scenarios. 
    """
    km, fm = np.meshgrid(ks, fs)
    vm_dt = np.vstack([sp.evaluate(ks, fut, max(t-dt, 1e-6)) for sp, fut in zip(spline_fs, fs)])
    pm_dt = np.vectorize(_b76_price)(vm_dt, km, 0, max(t-dt, 1e-6), max(t-dt, 1e-6), fm, 0, 0)
    p0 = np.vectorize(_b76_price)(curr_spline.evaluate(ks, f0, t), ks, 0, t, t, f0, 0, 0)
    
    f0_index = np.abs(fs-f0).argmin()
    delta = 0.5*(pm_dt[f0_index+1,:]-pm_dt[f0_index,:])/(fs[f0_index+1]-f0) + 0.5*(pm_dt[f0_index-1,:]-pm_dt[f0_index,:])/(fs[f0_index-1]-f0)
    theta = pm_dt[f0_index,:]-p0
    dpm_unhedged = pm_dt-p0
    dpm = dpm_unhedged-(fm-f0)*delta 
    return dpm, dpm_unhedged, delta, theta

In [30]:
# Get predicted price changes (hedged or unhedged), deltas, and thetas for each strike
dpm, dpm_u, delta, theta = gen_dpm(opt_ks, f, t, sp_cs, sp_fs, fs)
p0 = np.vectorize(_b76_price)(opt_vols, opt_ks, 0, t, t, f, 0, 0)

# Calculate EV by integrating over distribution of futures
ev = np.array([trapz(np.interp(xn, fs, dpm[:,i]) * full_dist, xn) for i in range(len(opt_ks))])

In [31]:
# Covariance Matrix generation for vol
def gen_vol_corr(zs, ts, corr_alpha=5.0, corr_exponent=2.0, corr_time_factor=0.2):
    z_diffs = zs[None, :] - zs[:, None]
    z_corr = np.exp(-np.abs(z_doffs)**corr_exponent/(2*corr_alpha**2))
    log_tte = np.log(ts)
    tte_diffs = log_tte[None, :] - log_tte[:, None]
    tte_corr = corr_time_factor * np.absolute(tte_diffs) + 1.0
    return z_corr / tte_corr

def gen_vol_std(zs, ts, vs, v0, std_center_sqrt_tte_factor=0.01, std_center_tte0=1e-5, std_ratios_zfactor=0.08,\
               std_ratios_zmin=0.5, std_ratios_min=0.97):
    def ratio_func(z):
        return std_ratios_zfactor * (z-std_ratios_zmin)**2 + std_ratios_min
    return vs * ratio_func(zs) * std_center_sqrt_tte_factor / np.sqrt(ts + std_center_tte0)

# Example linear optimization on calculating optimal position (no risk covariance penalization yet)

In [32]:
# Optimization
def pulp_optimizer(ev, theta, dpm, opt_params):
    max_size = opt_params['max_strike_position']
    max_theta = opt_params['max_strike_theta']
    max_loss = opt_params['max_loss']
    # Construct a linear programming maximization problem; objective: Maximize total EV
    prob = LpProblem('ev_opt', LpMaximize)
    var = range(len(ev))
    
    #Define weight variable (var) for each strike, constraint: abs(weight) on each strike less than max_size
    ws_vars = LpVariable.dicts('ws', var, -max_size, max_size)
    
    #Objective function: maximize sun of ev * weight on each strike
    prob += LpAffineExpression([(ws_vars[i], ev[i]) for i in var])
    
    #Set up constraints for optimization
    #Total theta across nodes == 0
    #For each strike:
    # ev (dpm) * weight >= max_loss
    # -max_theta <= theta * weight <= max_theta
    prob += LpConstraint(LpAffineExpression([(ws_vars[i], theta[i]) for i in var]), sense=0, rhs=0)
    for i in range(dpm.shape[0]):
        prob += LpConstraint(LpAffineExpression([(ws_vars[j], dpm[i][j]) for j in var]), sense=1, rhs=max_loss)
    for i in var:
        exp = LpAffineExpression([(ws_vars[i], theta[i])])
        prob += LpConstraint(exp, sense=1, rhs=-max_theta)
        prob += LpConstraint(exp, sense=-1, rhs=max_theta)
    prob.solve()
    
    #Retrieve optimal weights, optimization status, and objective function value
    ws = np.array([ws_vars[i].value() for i in var])
    opt_status = prob.status
    opt_ev = value(prob.objective)
    
    return ws, opt_status, opt_ev

In [33]:
# Optimization Parameters
opt_params = {
    'max_strike_position': 2000,
    'max_strike_theta': 100,
    'max_loss': -100
}
    
ws, opt_status, opt_ev = pulp_optimizer(ev, theta, dpm, opt_params)

# Outcome optimal positions

In [34]:
fig = figure(width=600, height=300, x_axis_label='strike', y_axis_label='i-vol', toolbar_sticky = False)
fig.line(x=ks, y=cs(ks), legend='Vol Curve', color='green', line_width=2)
fig.y_range = Range1d(0.05, .25)
fig.extra_y_ranges = {"position": Range1d(start=-100, end=100)}
fig.add_layout(LinearAxis(y_range_name="position"), 'right')
fig.scatter(x=opt_ks, y=ws, legend='Optimal Position', color='red', line_width=2, y_range_name="position")
fig.legend.location = 'top_right'
show(fig)

In [35]:
index = np.where((fs >= 2400) & (fs <= 3100))[0]
fig = figure(width=600, height=300, x_axis_label='Future scenario', y_axis_label='Theoretical Payoff')
fig.line(x=fs[index], y=(ws @ dpm[index].T), legend='Future Distribution', color='purple', line_width=2)
fig.legend.location = 'bottom_right'
show(fig)